In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient
import random, time
from datetime import datetime
import pandas as pd
import json

# A random programmatic shadow client ID.
SHADOW_CLIENT = "myShadowClient"

# The unique hostname that &IoT; generated for 
# this device.
HOST_NAME = "a1993mscwhaour-ats.iot.us-east-2.amazonaws.com"

# The relative path to the correct root CA file for &IoT;, 
# which you have already saved onto this device.
ROOT_CA = "AmazonRootCA1.pem"

# The relative path to your private key file that 
# &IoT; generated for this device, which you 
# have already saved onto this device.
PRIVATE_KEY = "47f811a754-private.pem.key"

# The relative path to your certificate file that 
# &IoT; generated for this device, which you 
# have already saved onto this device.
CERT_FILE = "47f811a754-certificate.pem"

# A programmatic shadow handler name prefix.
SHADOW_HANDLER = "A0138405J"

# Automatically called whenever the shadow is updated.
def myShadowUpdateCallback(payload, responseStatus, token):
  print()
  print('UPDATE: $aws/things/' + SHADOW_HANDLER + 
    '/shadow/update/#')
  print("payload = " + payload)
  print("responseStatus = " + responseStatus)
  print("token = " + token)

# Create, configure, and connect a shadow client.
print('Connecting to AWS IOT!')
myShadowClient = AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME, 8883)
myShadowClient.configureCredentials(ROOT_CA, PRIVATE_KEY,
  CERT_FILE)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(5)
myShadowClient.connect()

# Create a programmatic representation of the shadow.
myDeviceShadow = myShadowClient.createShadowHandlerWithName(
  SHADOW_HANDLER, True)

    
data = pd.read_csv('train_FD001.txt', delim_whitespace=True, header=None)
sensor_name = ['s'+str(i) for i in range(1,22)]
all_sensor_name = ['id', 'te', 'os1', 'os2', 'os3'] + sensor_name
data = pd.DataFrame(data.values, columns=all_sensor_name)
data['id'] = data['id'].apply(lambda s: 'FD001' + str(s))
for i in range(5000):
    array_data = data.iloc[i]
    time_now = datetime.utcnow()
    array_data = array_data.append(pd.Series([str(time_now),'A0138405J'], index = ['UTC_Timestamp', 'Matrix_Number']))
    array_data = array_data.to_dict()
    jsonPayload = json.dumps(array_data)
    print(jsonPayload)
    myDeviceShadow.shadowUpdate(str(jsonPayload),myShadowUpdateCallback,5)
    time.sleep(0.5)    